<a href="https://colab.research.google.com/github/bashirnubtk/Virtual-CPU-Emulator/blob/main/Final%20submission/Final_Virtual_Cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import logging

class VirtualCPU:
    def __init__(self):
        """ Initialize CPU with registers, memory, and program counter """
        self.registers = [0] * 16  # 16 General-purpose registers
        self.memory = [0] * 256  # 256-byte memory space
        self.pc = 0  # Program counter
        self.running = True  # CPU running state

        # Logging setup for debugging
        logging.basicConfig(filename="cpu_log.txt", level=logging.INFO, format="%(message)s")

    def fetch(self):
        """ Fetch the next instruction from memory """
        if self.pc < len(self.memory):
            instruction = self.memory[self.pc]
            self.pc += 1
            return instruction
        return None

    def decode(self, instruction):
        """ Decode the fetched instruction """
        return instruction  # Simple pass-through for now

    def execute(self, instruction):
        """ Execute the fetched instruction and log in binary format """
        if instruction is None:
            return

        opcode = instruction[0]
        binary_op = ""

        if opcode == "INPUT":
            reg = instruction[1]
            value = int(input(f"Enter value: "))
            self.registers[reg] = value
            binary_op = f"{bin(value)[2:].zfill(8)}"
            print(f"INPUT: R{reg} = {value} (Binary: {binary_op})")

       elif opcode == "OUTPUT":
    # For the OUTPUT opcode
    reg = instruction[1]  # Select the register
    value = self.registers[reg]  # Get the value from the selected register
    binary_op = f"{bin(value)[2:].zfill(8)}"  # Convert the value to binary format
    print(f"OUTPUT: R{reg} = {value} (Binary: {binary_op})")  # Print the output value in both decimal and binary

elif opcode == "ADD":
    # For the ADD opcode
    reg1, reg2, reg3 = instruction[1], instruction[2], instruction[3]  # Select three registers
    self.registers[reg1] = self.registers[reg2] + self.registers[reg3]  # Add the values of reg2 and reg3, store in reg1
    binary_op = f"{bin(self.registers[reg1])[2:].zfill(8)}"  # Convert the result to binary format
    print(f"ADD: R{reg1} = R{reg2} ({bin(self.registers[reg2])[2:].zfill(8)}) + R{reg3} ({bin(self.registers[reg3])[2:].zfill(8)}) → {self.registers[reg1]} (Binary: {binary_op})")  # Print the addition result and its binary representation

elif opcode == "SUB":
    # For the SUB opcode
    reg1, reg2, reg3 = instruction[1], instruction[2], instruction[3]  # Select three registers
    self.registers[reg1] = self.registers[reg2] - self.registers[reg3]  # Subtract the value of reg3 from reg2, store in reg1
    binary_op = f"{bin(self.registers[reg1])[2:].zfill(8)}"  # Convert the result to binary format
    print(f"SUB: R{reg1} = R{reg2} ({bin(self.registers[reg2])[2:].zfill(8)}) - R{reg3} ({bin(self.registers[reg3])[2:].zfill(8)}) → {self.registers[reg1]} (Binary: {binary_op})")  # Print the subtraction result and its binary representation

elif opcode == "MULT":
    # For the MULT opcode
    reg1, reg2, reg3 = instruction[1], instruction[2], instruction[3]  # Select three registers
    self.registers[reg1] = self.registers[reg2] * self.registers[reg3]  # Multiply the values of reg2 and reg3, store in reg1
    binary_op = f"{bin(self.registers[reg1])[2:].zfill(8)}"  # Convert the result to binary format
    print(f"MULT: R{reg1} = R{reg2} ({bin(self.registers[reg2])[2:].zfill(8)}) * R{reg3} ({bin(self.registers[reg3])[2:].zfill(8)}) → {self.registers[reg1]} (Binary: {binary_op})")  # Print the multiplication result and its binary representation

elif opcode == "HALT":
    # For the HALT opcode
    self.running = False  # Stop the execution
    print("HALT: Stopping execution")  # Print the message to indicate execution is stopped
    binary_op = "00000000"  # Set binary operation to zero (no operation)

else:
    # If the opcode is unknown
    print(f"Unknown opcode: {opcode}")  # Print an error message for unknown opcode

# Log the binary representation of the executed instruction
logging.info(f"Executed: {instruction} -> Binary: {binary_op}")

def run(self, program):
    """ Load and execute a program """
    self.memory[:len(program)] = program  # Load the program into memory
    while self.running:
        instruction = self.fetch()  # Fetch the instruction
        decoded_instr = self.decode(instruction)  # Decode the instruction
        self.execute(decoded_instr)  # Execute the decoded instruction

    # Print the final state of the registers
    print(f"Registers: {self.registers}")  # Print the final state of all registers



# Example Program (With MULT, ADD, SUB)
program = [
    ("INPUT", 0),   # Take input for R0
    ("INPUT", 1),   # Take input for R1
    ("MULT", 2, 0, 1),  # R2 = R0 * R1
    ("OUTPUT", 2),  # Print R2
    ("ADD", 3, 0, 1),  # R3 = R0 + R1
    ("OUTPUT", 3),  # Print R3
    ("SUB", 4, 0, 1),  # R4 = R0 - R1
    ("OUTPUT", 4),  # Print R4
    ("HALT",)       # Stop execution
]

# Run the Virtual CPU
cpu = VirtualCPU()
cpu.run(program)




Enter value: 7
INPUT: R0 = 7 (Binary: 00000111)
Enter value: 5
INPUT: R1 = 5 (Binary: 00000101)
MULT: R2 = R0 (00000111) * R1 (00000101) → 35 (Binary: 00100011)
OUTPUT: R2 = 35 (Binary: 00100011)
ADD: R3 = R0 (00000111) + R1 (00000101) → 12 (Binary: 00001100)
OUTPUT: R3 = 12 (Binary: 00001100)
SUB: R4 = R0 (00000111) - R1 (00000101) → 2 (Binary: 00000010)
OUTPUT: R4 = 2 (Binary: 00000010)
HALT: Stopping execution
Registers: [7, 5, 35, 12, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
